In [1]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import pulp
import pandas as pd
import random
import math
import matplotlib.pyplot as plt
from gurobipy import *

In [2]:
# 常数

v = 2  # lower limit of m
M = 10  # upper limit of m
t = 2  # lower limit of theta
N = 20  # upper limit of theta
rd = 1.2*1024#kb/s  # the downlink transmission rate
ru = 1.3*1024#kb/s  # the uplink transmission rate
O = 0.5*1024#kb  # the verification feedback size
B = 1#kb  # the transaction size
psi = 0.5  # a predefined parameter that can be defined leveraging the statistics on previous processes of block verification

In [3]:
# simulate的

Xi = np.random.uniform(10**3*8/128, 10**6*8/128, M)  # the amount of available computational resources at verifier i 

K = np.random.uniform(5, 500)  # the required computational resources for block verification task
kappa = np.random.uniform(1, 10)  # a coefficient given by the system
q = np.random.uniform(2, 10)  # an indicator factor representing the network scale
rho = np.random.normal(100, 5, M)  # the payment from verifier i to a cloud service provider
ci = np.multiply(rho, Xi)  #  the computational cost of verifier i

### 智能算法

### PSO

##### 初始化粒子群

In [4]:
# def population_pso(population0):

#     size = np.shape(population0)[0]
    
#     population_t = np.zeros((size, 12))
    
#     for i in range(size):
        
#         for j in range (12):
            
#             if j <= 1:
#                 population_t[i][j] = population0[i][j]
#             else:
#                 population_t[i][j] = population0[i][2][j-2]
                
#     return population_t

In [5]:
def species_origin_pso(population_size,v,M,t,N):
    
    population = []
    temporary = []
    
    for i in range(population_size):
        t3 = np.random.randint(0,2,M)
        t1 = sum(t3)
        t2 = random.randint(t,N)
        temporary = [t1,t2,t3]
        population.append(temporary)
        
#     for j in range(population_size):
#         while sum(population[j][2]) == 0:
#             population[j][2] = np.random.randint(0,2,M)
            
#     population_pso(population)
    
        
    return population

In [6]:
def population_pso(population0):

    size = np.shape(population0)[0]
    
    population = np.zeros((size, 2+M))
    
    for i in range(size):
        
        for j in range (2+M):
            
            if j <= 1:
                population[i][j] = population0[i][j]
            else:
                population[i][j] = population0[i][2][j-2]
                
    return population

In [7]:
def origin(population_size,v,M,t,N):
    return population_pso(species_origin_pso(population_size,v,M,t,N))

##### 目标函数

In [8]:
# beta1,2,3: weights;
# m: the number of the selected verifiiers
# theta: the number of transactions per block
# z: 0-1 variable

# beta1,2,3: weights;
# m: the number of the selected verifiiers
# theta: the number of transactions per block
# z: 0-1 variable


def  denominator0(z,Xi):  # 解决加入0-1变量后分母为0的情况
    
    maxi = []
    tem = 0
    
    for i in range(M):
        
        if z[i] == 0:
            tem = 0
            maxi.append(tem)
        else:
            tem = K*(1/(z[i]*Xi[i]))
            maxi.append(tem)
            
    return np.max(maxi)
        

def obj_fun_GA(beta1,beta2,beta3,m,theta,z):
    
    L = (theta*B/rd)+denominator0(z,Xi)+(psi*theta*B*m)+(O/ru)  # Latency
    S = kappa*m**q  # Secuity
    ci_sel = z*ci
    C = np.sum(ci_sel)/theta  # Cost
    
    L_m = (N*B/rd)+denominator0(z,Xi)+(psi*N*B*M)+(O/ru)
    S_m = kappa*M**q
    C_m = ci_sel.sum()/t
    
    u = beta1*(L/L_m)+beta2*((S_m-S)/S_m)+beta3*(C/C_m)
    return u

In [9]:
def ofunction(population):
    
    f = 0
    obj_function = []
    size = np.shape(population)[0]
    
    beta1 = 1/3
    beta2 = 1/3
    beta3 = 1/3
    
    for i in range(size):
        
        m = population[i][0]
        theta = population[i][1]
        z = population[i][2:12]
        
        f = obj_fun_GA(beta1,beta2,beta3,m,theta,z) 
        
        obj_function.append(f)
        
    return obj_function

##### 更新速度

In [10]:
# 更新速度，根据公式 V(t+1) = w*V(t) + c1*r1*(pbest_i-xi) + c2*r2*(gbest_xi)

def velocity_update(population_size,V,X,pbest,gbest,c1,c2,w,max_val):
    
    # 个体
    r11 = np.random.randint(-1,2,(population_size,2))  # (population_size,2)  # -1/0/1
    r21 = np.random.randint(-1,2,(population_size,2))  # (population_size,2)  # -1/0/1
    
    # 群体
    r12 = np.random.randint(-1,2,(population_size,dim-2))  # (population_size,10)  # -1/0/1
    r22 = np.random.randint(-1,2,(population_size,dim-2))  # (population_size,10)  # -1/0/1
    
    # 个体
    r1 = np.hstack((r11,r12))  # (population_size,12)
    
    # 群体
    r2 = np.hstack((r21,r22))  # (population_size,12)
    
    V = (w*V) + (c1*r1*(pbest-X)) + (c2*r2*(gbest-X)) #  注意这里得到的是一个矩阵
    
    # w: (1,1) 惯性常数
    # V: (population_size,12) 速度
    
    # c1: (1,1) 个体学习系数
    # r1: (population_size,1) 个体学习随机数
    # pbest: (population_size,12) 个体理想点
    # X: (population_size,12) 当前位置
    
    # c2: (1,1) 群体学习系数
    # r2: (population_size,1) 群体学习随机数
    # gbest: (population_size,)变(population_size,12) 群体理想点 —— 为了满足矩阵减法
    # X: (population_size,12) 当前位置
    
    # r1: (population_size,1)
    # r2: (population_size,1)
    
    # 这里是一个防止速度过大的处理，怕错过最理想值
    V[V <- max_val] = -max_val
    V[V >- max_val] = max_val
    
    return V

##### Test 速度更新参数

In [11]:
w = 1  # 设置惯性权重
c1 = 1  # 设置个体学习系数
c2 = 1  # 设置全局学习系数
M = 10
dim = 1+1+M  # 每个个体变量的数量(m+theta+M个0-1变量)
population_size = 100  # 初始化粒子群数量n
iter_num = 10  # 迭代次数
max_val = 3  # 限定最大速度为3
best_fitness = float(9e10)  # 初始化适应度的值
fitness_val_list = []
    
beta1 = 1/3
beta2 = 1/3
beta3 = 1/3
    
# 初始化各个粒子的位置
X = origin(population_size,v,M,t,N) # (population_size,2+M)
    
# 初始化各个粒子的速度(在最大限速范围内随机生成20个个体12个方向的速度)
V = np.random.uniform(-0.5,0.5,size=(population_size,dim))  # (population_size,2+M)
    
for i in range(population_size):
    for j in range(dim):
        if V[i,j] > 0:
            V[i,j] = 1;
        else:
            V[i,j] = 0;
    
p_fitness = ofunction(X)  # 初始化各个个体适应度的最优值
g_fitness = np.array(p_fitness).min()  # 初始化全局最理想适应度的最优值
fitness_val_list.append(g_fitness)  # 初始化每次迭代的群体最优函数值
    
pbest = X  # 初始化个体的最优位置
gbest = [X[np.array(p_fitness).argmin()][0:population_size]]  # 初始化整个整体的最优位置

r11 = np.random.randint(-2,2,(population_size,2))
r21 = np.random.randint(-2,2,(population_size,2))
r12 = np.random.randint(-1,1,(population_size,dim-2))
r22 = np.random.randint(-1,1,(population_size,dim-2))
r1 = np.hstack((r11,r12))
r2 = np.hstack((r21,r22))

V00 = (w*V) + (c1*r1*(pbest-X)) + (c2*r2*(gbest-X)) #  注意这里得到的是一个矩阵

/Users/l/miniforge3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:1970: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


w*V

c1*r1*(pbest-X)

c2*r2*(gbest-X)

X

V

X+V

##### 更新粒子位置

In [12]:
# 更新粒子位置，根据公式 X(t+1) = X(t) + Vt  (where t = 1)

def position_updata(population_size,X,V,v,M,t,N):
    
    dim = 2 + M
    
    XV = X + V
    
    for i in range(population_size):
        for j in range(2,dim):
            if XV[i][j] > 1:
                XV[i][j] = 1
            elif XV[i][j] < 0:
                XV[i][j] = 0
                
    for i in range(population_size):
        if XV[i][0] > M:
            XV[i][0] = M
        elif XV[i][0] < v:
            XV[i][0] = v
                
    for i in range(population_size):
        if XV[i][1] > N:
            XV[i][1] = N
        elif XV[i][1] < t:
            XV[i][1] = t

    return XV

##### PSO主体代码

In [13]:
# origin(population_size,v,M,t,N)
# ofunction(population)
# velocity_update(population_size,V,X,pbest,gbest,c1,c2,w,max_val)
# position_updata(population_size,X,V,v,M,t,N)

In [14]:
w = 1  # 设置惯性权重
c1 = 1  # 设置个体学习系数
c2 = 1  # 设置全局学习系数
r1 = None
r2 = None
M = 10
dim = 1+1+M  # 每个个体变量的数量(m+theta+M个0-1变量)
population_size = 100  # 初始化粒子群数量n
iter_num = 100  # 迭代次数
max_val = 3  # 限定最大速度为3
best_fitness = float(9e10)  # 初始化适应度的值
fitness_val_list = []
beta1 = 1/3
beta2 = 1/3
beta3 = 1/3

def pso(beta1,beta2,beta3,w,c1,c2,r1,r2,M,dim,population_size,iter_num,max_val,best_fitness,fitness_val_list):

    
    # 初始化各个粒子的位置
    X = origin(population_size,v,M,t,N) # (population_size,12)
    
    # 初始化各个粒子的速度(在最大限速范围内随机生成20个个体12个方向的速度)
    V = np.random.uniform(-0.5,0.5,(population_size,dim))  # (population_size, 12)
    
    for i in range(population_size):
        for j in range(dim):
            if V[i,j] > 0:
                V[i,j] = 1;
            else:
                V[i,j] = 0;
    
    p_fitness = ofunction(X)  # 初始化各个个体适应度的最优值
    g_fitness = np.array(p_fitness).min()  # 初始化全局最理想适应度的最优值
    fitness_val_list.append(g_fitness)  # 初始化每次迭代的群体最优函数值
    
    pbest = X  # 初始化个体的最优位置
    gbest = [X[np.array(p_fitness).argmin()][0:population_size-1]]  # 初始化整个整体的最优位置
    
    # 迭代
    for i in range(1,iter_num):
        V = velocity_update(population_size,V,X,pbest,gbest,c1,c2,w,max_val)  # 更新速度
        X = position_updata(population_size,X,V,v,M,t,N)  # 更新位置
        p_fitness2 = np.array(ofunction(X)) # 更新后群体中各个个体对应的函数值
        g_fitness2 = p_fitness2.min() # 更新后群体中的最优函数值
        
        # 更新每个粒子的历史的最优位置
        for j in range(population_size):
        
            # 如果原来某个个体对应的函数值大于更新后的——更新个体最优解和最优函数值
            if p_fitness[j] > p_fitness2[j]:
                pbest[j] = X[j]
                p_fitness[j] = p_fitness2[j]
            
            # # 如果原来群体对应的函数值大于更新后的——更新群体最优解和最优函数值
            if g_fitness > g_fitness2:
                gbest = X[p_fitness2.argmin()]
                g_fitness = g_fitness2
                
            fitness_val_list.append(g_fitness) # 加入本次迭代的群体最优函数值
            
            i += 1

    print(fitness_val_list[-1])
    print(gbest)
#     print("最优值是：%.5f" % fitness_val_list[-1])
#     print("最优解是：m=%.5f, theta=%.5f" % (gbest[0][0],gbest[1][0]))
    
#     for i in range(dim-2):
#         print("最优解是：z%d=%d" % (i+1,gbest[0][i+2]))

In [15]:
# Xi = np.random.uniform(10**3*8/128, 10**6*8/128, M)  # the amount of available computational resources at verifier i 

# if __name__ == '__main__':
#     pso()

##### 

##### PSO Example

In [16]:
# #f(x,y) = x^2 + y^2 + x
# import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib import cm
# from mpl_toolkits.mplot3d import Axes3D#导入该函数是为了绘制3D图
# import matplotlib as mpl
 
 
# #########
# #将数据绘图出来
# #生成X和Y的数据
# X = np.arange(-5,5,0.1) #-5到5的等距数组，距离为0.1，注意区分开range(),它返回的是一个列表
# Y = np.arange(-5,5,0.1)
# X,Y = np.meshgrid(X,Y)#该函数用来生成网格点坐标矩阵。
 
# #目标函数
# Z = X**2 + Y**2 + X
 
# #绘图
# fig = plt.figure()#创立一个画布
# ax = Axes3D(fig)#在这个画布里，生成一个三维的空间
# surf = ax.plot_surface(X,Y,Z,cmap=cm.coolwarm)#该函数是为了将数据在这三维空间里可视化出来。
# plt.show()
# ###########
 
# #######
# #计算适应度，这里的适应度就是我们目标函数Z的值，因为我们要求Z的最小值。
# #这两个函数，一般使用mpl画图的时候都会用到
# mpl.rcParams['font.sans-serif'] = ['SimHei']# 指定默认字体
# mpl.rcParams['axes.unicode_minus'] = False # 解决保存图像是负号'-'无法显示的问题
# #使用matplotliblib画图的时候经常会遇见中文或者是负号无法显示的情况
# #rcParams函数里的参数可以修改默认的属性，包括窗体大小、每英寸的点数、线条宽度、颜色、样式、坐标轴、坐标和网络属性、文本、字体等
 
# def fitness_func(X):
#     x = X[:,0]
#     y = X[:,1]
#     return x**2 + y**2 + x
# ##########
 
# #############
# #更新速度，根据公式V(t+1)=w*V(t)+c1*r1*(pbest_i-xi)+c1*r1*(gbest_xi)
# def velocity_update(V,X,pbest,gbest,c1,c2,w,max_val):
#     size = X.shape[0]#返回矩阵X的行数
#     r1 = np.random.random((size,1))#该函数表示成size行 1列的浮点数，浮点数都是从0-1中随机。
#     r2 = np.random.random((size,1))
#     V = w*V + c1*r1*(pbest-X)+c2*r2*(gbest-X)#注意这里得到的是一个矩阵
    
#     #这里是一个防止速度过大的处理，怕错过最理想值
#     V[V<-max_val] = -max_val
#     V[V>-max_val] = max_val
#     return V
# #########
 
 
# ########
# #更新粒子位置，根据公式X(t+1)=X(t)+V
# def position_updata(X,V):
#     return X+V
# ##########
 
 
# #########
# def pos():
#     w = 1 #设置惯性权重
#     c1 = 2 #设置个体学习系数
#     c2 = 2 #设置全局学习系数
#     r1 = None
#     r2 = None
#     dim = 2
#     size = 20 #这里是初始化粒子群，20个
#     iter_num = 1000 #迭代1000次
#     max_val = 0.5 #限定最大速度为0.5
#     best_fitness = float(9e10) #初始化适应度的值
#     fitness_val_list = []
    
#     #初始化各个粒子的位置
#     X = np.random.uniform(-5,5,size=(size,dim))
#     #初始化各个粒子的速度
#     V = np.random.uniform(-0.5,0.5,size=(size,dim))
    
#     p_fitness = fitness_func(X)#得到各个个体的适应度值
#     g_fitness = p_fitness.min()#全局最理想的适应度值
#     fitness_val_list.append(g_fitness)
    
#     pbest = X#初始化个体的最优位置
#     gbest = X[p_fitness.argmin()]#初始化整个整体的最优位置
    
#     #迭代
#     for i in range(1,iter_num):
#         V = velocity_update(V, X, pbest, gbest, c1, c2, w, max_val)
#         X = position_updata(X,V)
#         p_fitness2 = fitness_func(X)
#         g_fitness2 = p_fitness2.min()
        
#         #更新每个粒子的历史的最优位置
#         for j in range(size):
#             if p_fitness[j] > p_fitness2[j]:
#                 pbest[j] = X[j]
#                 p_fitness[j] = p_fitness2[j]
#             if g_fitness > g_fitness2:
#                 gbest = X[p_fitness2.argmin()]
#                 g_fitness = g_fitness2
                
#             fitness_val_list.append(g_fitness)
            
#             i += 1
            
#     print("最优值是：%.5f" % fitness_val_list[-1])
#     print("最优解是：x=%.5f,y=%.5f" % (gbest[0],gbest[1]))
    
    
#     plt.plot(fitness_val_list,c='r')
#     plt.title('迭代过程')
#     plt.show()
# #########
 
    
# if __name__ == '__main__':
#     pos()